### Dataset for the NHD to WQP

This script contains the definition of functions used in forming the <i>NHD</i> to <i>WQP</i> dataset
The <i>NHD</i> to <i>WQP</i> dataset consists of the following 6 csv files 

Listed below are the 6 csv files generated.

1. <b>WaterBody</b> - Table that stores details of Water Body
2. <b>BoundingBox</b> - Table that stores details about the bounding box 
3. <b>W2B</b> - Relation Table from Water Body to Bounding-Box 
4. <b>B2S</b> - Relation Table from Bounding-Box to Sites
5. <b>W2S</b> - Relation Table from Water Body to Sites
6. <b>Sites</b> - Table that stores details of the Sites


In [3]:
import shapefile
import pandas as pd
from shapely.geometry import *
from simpledbf import Dbf5
from pywqp import pywqp_client
from geopandas import *
import os,glob
import shutil
import pandas as pd

doing node 'org'
doing node 'station'
doing node 'result'
doing node 'activity'


In [4]:
# To avoid throwing of warnings
pd.options.mode.chained_assignment = None 

# Getting all the polygon shape files 
data_path = "NHD_High_Resolution/NHD_Minnesota/Shape/"
sf = shapefile.Reader(data_path+"NHDWaterbody.shx")
shapes = sf.shapes()

# Global variables intialization
df_sites = pd.DataFrame()
df_master_sites_table = pd.DataFrame()

-------------------------

#### Helper Functions that aid in the creation of the data-set

Below section of the notebook defines functions that aid in the creation of the dataset

1. Function to construct a polygon for water body with points
2. Function to query WQP with the bounding box coordinates
3. Function to check if the coordinates lie within the polygon of water body.

##### 1. Function to construct a polygon with points

In [2]:
# Construct polygon of the water body
def construct_polygon(points):
    """
        Constructs polygon of the water body
        Args:
            points (list): Input String
        Returns:
            polygon (shapely.geometry.polygon.Polygon)
    """
    polygon = shape(points)
    return polygon

##### 2. Function to query WQP on the bounding box coordinates

In [5]:
# Query WQP with the bounding box co-ordinates
def web_service_call(bbox):
    """
        Retrieves data from water quality portal using web service client
        Args:
            bbox (list): Input list
        Returns:
            result_df (pandas DataFrame)
    """
    client_instance = pywqp_client.RESTClient()
    bbox_params = ','.join(bbox)
    
    # Preparing the input payload
    verb = 'get'
    host_url = 'http://waterqualitydata.us'
    resource_label = 'station'
    params = {'bBox':bbox_params,'siteType':'Lake, Reservoir, Impoundment'}
    equivalent_url = client_instance.create_rest_url(host_url, resource_label, params, mime_type='text/csv')
    
    # Make the web-service call and get the response into a DataFrame
    result = client_instance.request_wqp_data(verb, host_url, resource_label, params, mime_type='text/csv')
    result_df = client_instance.response_as_pandas_dataframe(result)
    return result_df


##### 3. Function to check if the coordinates lie within the polygon of water body

In [3]:
# Checks presence of point inside the polygon
def is_point_in_polygon(point,polygon):
    """
        Checks presence of point inside the polygon
        Args:
            point (list): Input list, polygon (shapely.geometry.polygon.Polygon): Input polygon 
        Returns:
            bool
    """
    if polygon.contains(point):
        return True
    else:
        return False

---------------------

Below section of the notebook contains the definition of functions to create each of the 6 tables

##### 1) WaterBody - Table that stores details of water body

This table stores details of the water body from <i>NHD</i>. Attributes of this table are :

1. NHD_LAKE_ID - The ID given for the water body in <i>NHD</i> web-site
2. AREA(sqkm) - Area of the water body in sq kms
3. ELEVATION(feet) - Elevation of the water body in feet
4. SHAPE_LENG(decimaldegrees) - Length of the water body shape in decimal degrees
5. SHAPE_AREA(sqdecimaldegrees) - Area of the water body shape in square decimal degrees

In [5]:
def create_wb_table(df):
    """
        Creates the dataframe of details of water bodies
        Args:
            df (pandas dataframe): Input dataframe
        Returns:
            df_wb (pandas dataframe)
    """
    df_wb = df[['PERMANENT_','GNIS_NAME','GNIS_ID', 'AREASQKM', 'ELEVATION', 'FTYPE', 'FCODE', 'FDATE',
                           'SHAPE_LENG', 'SHAPE_AREA']]
    df_wb = df_wb.rename(index=str, columns={"PERMANENT_":"NHD_LAKE_ID",'AREASQKM':'AREA(sqkm)',
                                                     'ELEVATION':'ELEVATION(feet)','SHAPE_LENG':'SHAPE_LENG(decimaldegrees)',
                                                     'SHAPE_AREA': 'SHAPE_AREA(sqdecimaldegrees)'})
    return df_wb

##### 2) BoundingBox - Table that stores details about the bounding box 

This table stores details about the bounding boxes read from Shapefiles in <i>NHD</i>. Attributes of this table are:

1. BB_ID - ID for the water body bounding box
2. North - Minimum Latitude 
3. South - Maximum Latitude 
4. West - Minimum Longitude
5. East - Maximum Longitude


In [6]:
def create_bbox_table(df,shapes,start):
    """
        Creates the dataframe of details of bounding box of water body
        Args:
            df (pandas dataframe): Input dataframe, shapes (list): Input list,
            start (str): Input string
        Returns:
            df_bbox (pandas dataframe)
    """
    bbox_columns = ['BB_ID', 'North', 'South', 'West', 'East']
    df_bbox = pd.DataFrame(columns=bbox_columns)
    bbox_data = map(lambda (index,y): ('bb_'+str(index+start),y.bbox[1],y.bbox[3],y.bbox[0],y.bbox[2]), enumerate(shapes))
    df_bbox = pd.DataFrame(bbox_data,columns=bbox_columns,index=None)
    return df_bbox

##### 3) W2B - Relation Table from Water Body to Bounding-Box 

This table stores the relation between water body and bounding box. Attributes of this table are:

1. NHD_LAKE_ID - The ID given for the water body in NHD web-site
2. GNIS_NAME - Name of the water body as per the Geographic Name Information System  
3. BB_ID - ID for the water body bounding box


In [9]:
def create_w2b_table(df,shapes,df_bbox):
    """
        Creates the dataframe of relation of bounding box to water body
        Args:
            df (pandas dataframe): Input dataframe, 
            shapes (list): Input list,
            df_bbox (pandas dataframe): Input dataframe
        Returns:
            (df_w2b,w2b_table) (pandas dataframe, pandas dataframe)
    """
    # Columns for W2B table
    w2b_columns = ['NHD_LAKE_ID','BB_ID']
    df_w2b = pd.DataFrame(columns=w2b_columns)
    
    # Processes the NHD water body dataframe, picking required columns, indexing the dataframe
    df_v1 = pd.DataFrame(df[['PERMANENT_','GNIS_NAME']])
    df_v1['Index'] = range(len(df_v1))
    df_v1.set_index('Index')
    
    # Processes Bounding Box dataframe, indexing the dataframe
    df_bbox['Index'] = range(len(df_bbox))
    df_bbox.set_index('Index')
    
    # Joins the NHD water body dataframe and Bounding Box dataframe
    df_w2b = df_v1.merge(df_bbox,left_on = 'Index',right_on='Index')
    
    # Projects only required columns and renames the joined dataframe 
    df_w2b = df_w2b[['PERMANENT_','GNIS_NAME','BB_ID']]
    df_w2b = df_w2b.rename(index=str, columns={"PERMANENT_":"NHD_LAKE_ID"})
    
    # Reorders the columns in the joined dataframe
    w2b_table = df_w2b[w2b_columns]
    
    return (df_w2b,w2b_table)

##### 4) B2S - Relation Table from Bounding-Box to Sites 
This table stores the relation between water body and bounding box. Attributes of this table are:

1. BB_ID - ID for the water body bounding box
2. SITE_ID - ID for the site as per the <i>WQP</i>

##### 5) W2S - Relation Table from Water Body to Sites
This table stores the relation between water body and sites (obtained from <i>WQP</i>). Attributes of this table are:

1. NHD_LAKE_ID - The ID given for the water body in NHD web-site
2. GNIS_NAME - Name of the water body as per the Geographic Name Information System
3. SITE_ID - ID for the site as per the <i>WQP</i>
4. MonitoringLocationName - Name of the site as per <i>WQP</i> 
5. IsInsideLake - True/False depending on whether the site is inside the water body or not
6. DistToShore(m) - Distance of the site to the shore of the water body in meters

Inorder to obtain the b2s and w2s tables, following helper fucntions are used.<br> Following section of the notebook gives the function definitions of these helper functions.

1. Function to calculate distance of the sites from the water body polygon
2. Function to fetch the details of sites associated with the water body
3. Function to form the relation between the bounding, sites and water body

##### 1. Function to calculate distance of the sites from the water body polygon

In [7]:
def dist_in_poly(point,polygon):
    """
        Calculates distance of point from the boundary of the polygon
        Args:
            point (shapely.geometry.point.Point): Input Point, 
            polygon (shapely.geometry.polygon.Polygon): Input Polygon
        Returns:
            dist (float)
    """
    new_point = polygon.boundary.interpolate(polygon.boundary.project(point))
    dist = geopy.distance.vincenty((new_point.x,new_point.y),(point.x,point.y)).m
    dist = float("{0:.3f}".format(dist))
    return dist

In [11]:
# Function to check if all of the points lie at a distance of around 100 m from the boundary
# and calculate the distance

import geopy
from shapely.geometry import Polygon, Point, LinearRing
from geopy.distance import vincenty
def dist_out_poly(poly,point):
    """
        Check if the point lies at a distance of less than 100 m from the boundary
        of the polygon and calculate the distance
        Args:
            poly (shapely.geometry.polygon.Polygon): Input Polygon
            point (shapely.geometry.point.Point): Input Point, 
        Returns:
            distance_points (float)
    """
    pol_ext = poly.boundary
    d = pol_ext.project(point)
    p = pol_ext.interpolate(d)
    distance_points = geopy.distance.vincenty((point.x,point.y),(p.x,p.y)).m
    if distance_points<=100:
        return float("{0:.3f}".format(distance_points))
    else:
        return None

In [12]:
def dist_poly(lat_long,is_inside,polygon):
    """
        Returns distance of the point depending on whether it is inside or outside
        the water body polygon
        Args:
            lat_long (tuple): Input tuple
            is_inside (bool): Input bool, 
            polygon (shapely.geometry.polygon.Polygon): Input Polygon
        Returns:
            (float)
    """
    if is_inside:
        return dist_in_poly(lat_long,polygon)
    else:
        return dist_out_poly(polygon,lat_long)
        

##### 2. Function to fetch the details of sites associated with the water body

In [13]:
def validate_site_ids(df,polygon,bbox_id):
    """
        Checks if the site ids lies at a distance of less than 100 m 
        from the water body polygon and returns dataframe with site and whether it 
        lies inside/outside the polygon, its distance from it
        Args:
            df (pandas dataframe): Input dataframe,
            polygon (shapely.geometry.polygon.Polygon): Input Polygon,
            bbox_id (str): Input String
        Returns:
            df_sites_master_inner (pandas dataframe)
    """
    if (df is None) or df.empty:
        return
    
    # Initialize values
    site_ids_list = []
    extra_site_ids_list = []
    
    df['lat_long'] = tuple(zip(df['LongitudeMeasure'],df['LatitudeMeasure']))
    df['lat_long'] = df['lat_long'].apply(lambda lat_long_tuple: Point(lat_long_tuple[0], lat_long_tuple[1]))
    
    df['point_in_polygon'] = df['lat_long'].apply(lambda point: is_point_in_polygon(point, polygon))
    df['distance'] = df.apply(lambda row:dist_poly(row['lat_long'],row['point_in_polygon'],polygon),axis=1)
    
    df = df.dropna(subset=['distance'])
    df_sites_master_inner = df[['MonitoringLocationIdentifier','MonitoringLocationName','point_in_polygon','distance']]
    df_sites_master_inner['BB_ID'] = bbox_id
    return df_sites_master_inner

##### 3. Function to form the relation between the bounding, sites and water body.

In [14]:

def func_2(index,shape,start):
    """
       Helper function to produce relation between sites, water body and 
       bounding-box in dataframe
        Args:
            index (int): Input int,
            shape (shapely.geometry.polygon.ShapeFile): Input ShapeFile,
            start (int): Input int
    """
    # Get bounding box of the water body
    bbox = shape.bbox
    sites_b=[]
    
    # Global variables
    global df_sites,df_master_sites_table
    
    # Construct polygon
    polygon = construct_polygon(shape)
    bbox_list = [str(x) for x in bbox]
    bbox_id = 'bb_'+str(index+start)
    
    # Query WQP on boundary box coordinates (WSEN):
    df_sites_bbox = web_service_call(bbox_list)
    
    # Obtain the data of valid site IDs that lie within the lake polygon
    df_master_sites = validate_site_ids(df_sites_bbox,polygon,bbox_id)
    df_master_sites_table = df_master_sites_table.append(df_master_sites,ignore_index=True)
    
    # Obtain details of site IDs
    df_sites = df_sites.append(df_sites_bbox,ignore_index=True)

def create_b2s_table(shapes,w2b_master_table,start):
    """
        Produce relation between sites, water body and bounding-box in dataframe
        Args:
            shapes (list): Input list,
            w2b_master_table (pandas dataframe): Input dataframe,
            start (int): Input int
        Returns:
            b2s_table,w2s_table (pandas dataframe, pandas dataframe)
    """
    df_master_table = pd.DataFrame()
    
    # Columns for B2S table
    b2s_cols = ['BB_ID', 'SITE_ID']
    
    # Columns for W2S table
    w2s_cols = ['NHD_LAKE_ID', 'GNIS_NAME','SITE_ID','MonitoringLocationName','IsInsideLake','DistToShore(m)']
    
    # Produce relation between sites, water body and bounding-box
    map(lambda (index,y) : func_2(index,y,start), enumerate(shapes))
    
    # Form details about sites table
    df_master_table = df_master_sites_table.rename(index=str, columns={"MonitoringLocationIdentifier":"SITE_ID",'point_in_polygon':'IsInsideLake',
                                                     'distance':'DistToShore(m)'})
    # Join the W2B and df_master_table to produce relation between sites, water body, bounding box
    joined_master_table = df_master_table.merge(w2b_master_table,left_on = 'BB_ID',right_on='BB_ID')
   
    # Form the B2S table and W2S table with columns
    b2s_table = joined_master_table.loc[:,b2s_cols]
    w2s_table = joined_master_table.loc[:,w2s_cols]
    
    # Rename few column names in W2S table
    w2s_table = w2s_table.rename(index=str, columns={"GNIS_NAME":"GNIS_LAKE_NAME"})
    
    return b2s_table,w2s_table

##### 6) Sites table - Table that stores details of sites
This table stores the relation between water body and sites (obtained from WQP). It includes all the attributes as obtained from <i>WQP</i>

In [15]:
def create_sites_table(b2s_table):
    """
        Form sites table
        Args:
            b2s_table (pandas dataframe): Input pandas dataframe
    """
    global df_sites
    
    # Getting the unique site values for the dataframe
    site_vals = set(b2s_table['SITE_ID'].values.tolist())
    
    # list of column names needed in sites dataframe
    list_cols = ['OrganizationIdentifier','OrganizationFormalName','SITE_ID','MonitoringLocationName','MonitoringLocationTypeName',
     'MonitoringLocationDescriptionText','HUCEightDigitCode','DrainageAreaMeasure/MeasureValue','DrainageAreaMeasure/MeasureUnitCode',
     'ContributingDrainageAreaMeasure/MeasureValue','ContributingDrainageAreaMeasure/MeasureUnitCode',
     'LatitudeMeasure','LongitudeMeasure','SourceMapScaleNumeric','HorizontalAccuracyMeasure/MeasureValue',
     'HorizontalAccuracyMeasure/MeasureUnitCode','HorizontalCollectionMethodName',
     'HorizontalCoordinateReferenceSystemDatumName','VerticalMeasure/MeasureValue','VerticalMeasure/MeasureUnitCode',
     'VerticalAccuracyMeasure/MeasureValue','VerticalAccuracyMeasure/MeasureUnitCode',
     'VerticalCollectionMethodName','VerticalCoordinateReferenceSystemDatumName',
     'CountryCode','StateCode','CountyCode','AquiferName','FormationTypeText','AquiferTypeName','ConstructionDateText',
     'WellDepthMeasure/MeasureValue','WellDepthMeasure/MeasureUnitCode','WellHoleDepthMeasure/MeasureValue',
     'WellHoleDepthMeasure/MeasureUnitCode','ProviderName']
    
    # Renaming and reordering the column names
    df_sites = df_sites.rename(index=str, columns={"MonitoringLocationIdentifier":"SITE_ID"})
    df_sites = df_sites[df_sites['SITE_ID'].isin(site_vals)]
    
    # Putting SITE_ID in the beginning of the dataframe
    a, b = list_cols.index('SITE_ID'), list_cols.index('OrganizationIdentifier')
    list_cols[b], list_cols[a] = list_cols[a], list_cols[b]
    
    df_sites = df_sites.loc[:,list_cols]
    df_sites.drop(df_sites.columns[[34,35]], axis=1, inplace=True)

--------------------

Below is the definition of the function that calls the above discussed functions and creates the dataset of 6 tables

In [16]:
def create_6_tables_each_folder(folderpath):
    """
        Forms the data set for the nhd to wqp
        Args:
            folderpath (string): Input String
    """
    global shapes
    df = pd.read_csv(folderpath+'/lakes.csv')
    
    # Read the info file to get the start and end index for shapes
    df_info = pd.read_csv(folderpath+'/info.csv')
    start = df_info.loc[0]['start_index']
    #end = df_info.loc[0]['end_index']
    end =  df_info.loc[0]['start_index']+1000
    shapes = shapes[start:end]
    
    # Create tables data path within the folder
    table_data_path = folderpath+'/tables'
    if os.path.exists(table_data_path): 
        shutil.rmtree(table_data_path)
    os.makedirs(table_data_path)
    
    # Create 1. water body dataset
    wb_table = create_wb_table(df)
    wb_table.to_csv(table_data_path+'/waterbody.csv',index=None)
    
    # Create 2. bounding-box dataset
    bbox_table = create_bbox_table(df,shapes,start)
    bbox_table.to_csv(table_data_path+'/boundingbox.csv', index=None)
    
    # Create 3. water body to bounding-box (w2b) dataset
    w2b_master_table,w2b_table = create_w2b_table(df,shapes,bbox_table)
    w2b_table.to_csv(table_data_path+'/w2b.csv',index=None)
  
    # Create 4. bounding-box to site (b2s) & 5. water body to site (w2s) dataset
    b2s_table,w2s_table = create_b2s_table(shapes,w2b_master_table,start)
    b2s_table.to_csv(table_data_path+'/b2s.csv',index=None)

    #  5. water body to site dataset(w2s)
    w2s_table.to_csv(table_data_path+'/w2s.csv',index=None)
    
    # Create 6. sites dataset
    create_sites_table(b2s_table)
    df_sites.to_csv(table_data_path+'/sites.csv',index=None)
    

In [1]:
def final_func(folderpath_list):
    """
        Forms the data set for the nhd to wqp with 10 processes
        Args:
            folderpath_list (list): Input list
    """
    from multiprocessing import Pool
    pool = Pool(processes=10)
    pool.map(create_6_tables_each_folder,folderpath_list)
    

------------------